In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_3.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_0.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_2.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_15_buy2buy_v6_0.pqt
/kaggle/input/otto-pipeline2-lb-0-576/__results__.html
/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_1.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_0.pqt
/kaggle/input/otto-pipeline2-lb-0-576/submission.csv
/kaggle/input/otto-pipeline2-lb-0-576/__notebook__.ipynb
/kaggle/input/otto-pipeline2-lb-0-576/__output__.json
/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_1.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_2.pqt
/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_3.pqt
/kaggle/input/otto-pipeline2-lb-0-576/custom.css
/kaggle/input/otto-recommender-system/sample_submission.csv
/kaggle/input/otto-recommender-system/test.jsonl
/kaggle/input/otto-recommender-system/train.jsonl

In [2]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import  itertools

In [3]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}
def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob('../input/otto-chunk-data-inparquet-format/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

test_df = load_test()
print('Test data has shape',test_df.shape)
test_df.head()

Test data has shape (6928123, 4)


,session,aid,ts,type
0,13099779,245308,1661795832,0
1,13099779,245308,1661795862,1
2,13099779,972319,1661795888,0
3,13099779,972319,1661795898,1
4,13099779,245308,1661795907,0


In [4]:
%%time
VER=6
DISK_PIECES=4
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

top_20_clicks = pqt_to_dict(pd.read_parquet(f'/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_0.pqt'))
for k in range(1, DISK_PIECES):
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(f'/kaggle/input/otto-pipeline2-lb-0-576/top_20_clicks_v6_{k}.pqt')))
top_20_buys = pqt_to_dict( pd.read_parquet(f'/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_0.pqt'))
for k in range(1, DISK_PIECES):
    top_20_buys.update( pqt_to_dict( pd.read_parquet(f'/kaggle/input/otto-pipeline2-lb-0-576/top_15_carts_orders_v6_{k}.pqt')))
top_20_buy2buy = pqt_to_dict(pd.read_parquet(f'/kaggle/input/otto-pipeline2-lb-0-576/top_15_buy2buy_v6_0.pqt'))

top_clicks = test_df.loc[test_df['type']== 0, 'aid'].value_counts().index.values[:100]
top_carts = test_df.loc[test_df['type']== 1, 'aid'].value_counts().index.values[:100]
top_orders = test_df.loc[test_df['type']== 2, 'aid'].value_counts().index.values[:100]

print('Here are size of our 3 co-visitation matrices:')
print( len(top_20_clicks), len(top_20_buy2buy), len(top_20_buys))

Here are size of our 3 co-visitation matrices:
1837166 1168768 1837166
CPU times: user 2min 5s, sys: 12.2 s, total: 2min 17s
Wall time: 2min 18s


In [5]:
type_weight_multipliers = {0: 1, 1: 5, 2: 4}

def suggest_clicks(df):
    # USER HISTORY AIDS AND TYPES
    aids=df.aid.tolist()
    types = df.type.tolist()
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    # RERANK CANDIDATES USING WEIGHTS
    if len(unique_aids)>=100:
        weights=np.logspace(0.1,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        sorted_aids = [k for k,v in aids_temp.most_common(100)]
        return sorted_aids
    # USE "CLICKS" CO-VISITATION MATRIX
    aids2 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    aids3 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2+aids3).most_common(100) if aid2 not in unique_aids]    
    result = unique_aids + top_aids2[:100 - len(unique_aids)]
    # USE TOP20 TEST CLICKS
    tops = set(list(top_clicks)) - set(result)
    
    return result + list(tops)[:100-len(result)]

    


def suggest_carts(df):
    # User history aids and types
    aids = df.aid.tolist()
    types = df.type.tolist()
    
    # UNIQUE AIDS AND UNIQUE BUYS
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    df = df.loc[(df['type'] == 0)|(df['type'] == 1)]
    unique_buys = list(dict.fromkeys(df.aid.tolist()[::-1]))
    
    # Rerank candidates using weights
    if len(unique_aids) >= 100:
        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        
        # Rerank based on repeat items and types of items
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        
        # Rerank candidates using"top_20_carts" co-visitation matrix
        aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_buys if aid in top_20_buys]))
        for aid in aids2: aids_temp[aid] += 0.1
        sorted_aids = [k for k,v in aids_temp.most_common(100)]
        return sorted_aids
    
    # Use "cart order" and "clicks" co-visitation matrices
    aids1 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_aids if aid in top_20_buy2buy]))
    
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids1+aids2+aids3).most_common(100) if aid2 not in unique_aids] 
    result = unique_aids + top_aids2[:100 - len(unique_aids)]
    
    # USE TOP20 TEST ORDERS
    tops = set(list(top_carts)) - set(result)

    return result + list(tops)[:100-len(result)]


def suggest_buys(df):
    # USER HISTORY AIDS AND TYPES
    aids=df.aid.tolist()
    types = df.type.tolist()
    # UNIQUE AIDS AND UNIQUE BUYS
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    df = df.loc[(df['type']==1)|(df['type']==2)]
    unique_buys = list(dict.fromkeys( df.aid.tolist()[::-1] ))
    # RERANK CANDIDATES USING WEIGHTS
    if len(unique_aids)>=100:
        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        # RERANK CANDIDATES USING "BUY2BUY" CO-VISITATION MATRIX
        aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
        for aid in aids3: aids_temp[aid] += 0.1
        sorted_aids = [k for k,v in aids_temp.most_common(100)]
        return sorted_aids
    # USE "CART ORDER" CO-VISITATION MATRIX
    aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    # USE "BUY2BUY" CO-VISITATION MATRIX
    aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2+aids3).most_common(100) if aid2 not in unique_aids] 
    result = unique_aids + top_aids2[:100 - len(unique_aids)]
    # USE TOP20 TEST ORDERS
    tops = set(list(top_orders)) - set(result)

    return result + list(tops)[:100-len(result)]

In [6]:
%%time

pred_df_clicks = test_df.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: suggest_clicks(x)
)


CPU times: user 7min 35s, sys: 9.26 s, total: 7min 44s
Wall time: 7min 44s


In [7]:
pred_df_clicks

session
12899779    [59625, 1253524, 737445, 438191, 731692, 17907...
12899780    [1142000, 736515, 973453, 582732, 889686, 1502...
12899781    [918667, 199008, 194067, 57315, 141736, 146057...
12899782    [1007613, 595994, 1033148, 834354, 479970, 169...
12899783    [1817895, 607638, 1754419, 1216820, 1729553, 3...
                                  ...                        
14571577    [1141710, 1276792, 1004292, 1666114, 367734, 8...
14571578    [519105, 977826, 1811714, 822641, 815460, 1536...
14571579    [739876, 1209992, 1750859, 1550479, 51363, 785...
14571580    [202353, 1314576, 433425, 871658, 1231403, 925...
14571581    [1100210, 1684953, 462056, 1158237, 1401429, 6...
Length: 1671803, dtype: object

In [8]:
pred_df_clicks = pd.DataFrame(pred_df_clicks)
pred_df_clicks.reset_index(inplace=True)
pred_df_clicks = pred_df_clicks.rename(columns={0:'aids'})
pred_df_clicks = pred_df_clicks.explode('aids')
pred_df_clicks['aids'] = pred_df_clicks['aids'].astype('int32')
pred_df_clicks['session'] = pred_df_clicks['session'].astype('int32')
pred_df_clicks.to_parquet('pred_df_clicks.pqt')

import gc
del pred_df_clicks
_ = gc.collect()

In [9]:
%%time
pred_df_carts = test_df.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: suggest_carts(x)
)

CPU times: user 34min 13s, sys: 24.2 s, total: 34min 37s
Wall time: 34min 30s


In [10]:
pred_df_carts = pd.DataFrame(pred_df_carts)
pred_df_carts.reset_index(inplace=True)
pred_df_carts = pred_df_carts.rename(columns={0:'aids'})
pred_df_carts = pred_df_carts.explode('aids')
pred_df_carts['aids'] = pred_df_carts['aids'].astype('int32')
pred_df_carts['session'] = pred_df_carts['session'].astype('int32')
pred_df_carts.to_parquet('pred_df_carts.pqt')

del pred_df_carts
_ = gc.collect()

In [11]:
%%time
pred_df_buys = test_df.sort_values(["session", "ts"]).groupby(["session"]).apply(
    lambda x: suggest_buys(x)
)

CPU times: user 31min 51s, sys: 29.5 s, total: 32min 21s
Wall time: 32min 14s


In [12]:
pred_df_buys = pd.DataFrame(pred_df_buys)
pred_df_buys.reset_index(inplace=True)
pred_df_buys = pred_df_buys.rename(columns={0:'aids'})
pred_df_buys = pred_df_buys.explode('aids')
pred_df_buys['aids'] = pred_df_buys['aids'].astype('int32')
pred_df_buys['session'] = pred_df_buys['session'].astype('int32')
pred_df_buys.to_parquet('pred_df_buys.pqt')